In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
DIR_INPUT_MINE_SUB = '../input/hubmap-pred-stitch-test-transfer-tune-enc-weighted'
DIR_INPUT_PROJ = '/kaggle/input/hubmap-kidney-segmentation'
DIR_INPUT_REF_SUB = '../input/global-mask-shift'
import os
for dirname, _, filenames in os.walk(DIR_INPUT_MINE_SUB):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_mine_sub = pd.read_csv(os.path.join(DIR_INPUT_MINE_SUB, 'submission.csv'), index_col=0)
df_ref_sub = pd.read_csv(os.path.join(DIR_INPUT_REF_SUB, 'submission.csv'), index_col=0)

In [ ]:
from keras import backend as K

smooth = 1

def dice_coef_bin(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_pred_f = K.greater(y_pred_f, 0.5)
    y_pred_f = K.cast(y_pred_f, y_true_f.dtype)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

import tifffile
def read_raw_tiff(img_path, mask_bin=None):
    img_raw = tifffile.imread(img_path)
    if len(img_raw.shape) > 3:
        img_raw = np.transpose(img_raw.squeeze(), (1,2,0))
    if mask_bin is None:
        return img_raw
    else:
        assert img_raw.shape[:2] == mask_bin.shape[:2]
    return img_raw


def get_mask_bin(rle_raw, n_pix_height, n_pix_width):
    '''convert the raw rle data of one image into a binary mask'''
    rle_list = np.array(rle_raw.split(' ')).astype(int)
    rle_paired = np.array([rle_list[::2] - 1, rle_list[1::2]]).T
    mask_bin = np.zeros(n_pix_width * n_pix_height)
    for start, run_length in rle_paired:
        mask_bin[start: start + run_length] = 1
    mask_bin = mask_bin.reshape(n_pix_width, n_pix_height).T.astype(int).reshape(n_pix_height, n_pix_width, 1)
    mask_bin = mask_bin.astype(np.uint8)
    return mask_bin

In [ ]:
import cv2
from tqdm.auto import tqdm
for img_whole_id in tqdm(df_mine_sub.index):
    img_whole_filename = img_whole_id + '.tiff'
    im_whole_path = os.path.join(os.path.join(DIR_INPUT_PROJ, 'test',img_whole_filename))
    img_whole = read_raw_tiff(im_whole_path, mask_bin=None)
    img_whole_h, img_whole_w = img_whole.shape[:2]

    scale_factor = 0.25
    img_whole_h_scaled, img_whole_w_scaled = np.round(
        np.array([img_whole_h, img_whole_w])* scale_factor
    ).astype(int)
    img_whole_scaled = cv2.resize(
        img_whole.astype(np.uint8), 
        (img_whole_w_scaled, img_whole_h_scaled), 
        interpolation=cv2.INTER_NEAREST
    )
    del img_whole
    print(img_whole_id)
    print(img_whole_h, img_whole_w)
    
    mask_rle_str = df_mine_sub.loc[img_whole_id,'predicted']
    mask_bin = get_mask_bin(mask_rle_str,img_whole_h, img_whole_w).reshape((img_whole_h, img_whole_w)) 
    mask_bin_scaled_mine = cv2.resize(
        mask_bin.astype(np.uint8), 
        (img_whole_w_scaled, img_whole_h_scaled), 
        interpolation=cv2.INTER_NEAREST
    )
    del mask_bin
    
    mask_rle_str = df_ref_sub.loc[img_whole_id,'predicted']
    mask_bin = get_mask_bin(mask_rle_str, img_whole_h, img_whole_w).reshape((img_whole_h, img_whole_w))
    mask_bin_scaled_ref = cv2.resize(
        mask_bin.astype(np.uint8), 
        (img_whole_w_scaled, img_whole_h_scaled), 
        interpolation=cv2.INTER_NEAREST
    )
    del mask_bin
    print(
        dice_coef_bin(mask_bin_scaled_mine.astype(np.float32), mask_bin_scaled_ref.astype(np.float32))
    )
    
    cv2.imwrite(f'./{img_whole_id}_img_whole_scaled.tiff', img_whole_scaled)
    del img_whole_scaled
    cv2.imwrite(f'./{img_whole_id}_mask_bin_scaled_pred.tiff', mask_bin_scaled_mine*255)
    del mask_bin_scaled_mine
    cv2.imwrite(f'./{img_whole_id}_mask_bin_scaled_ref.tiff', mask_bin_scaled_ref*255)
    del mask_bin_scaled_ref
    
    
    